# Using LLMs to Generate Synthetic Data For Benchmarking

## Overview
This notebook generates diverse synthetic data examples to benchmark multiple open-source models for Named Entity Recognition (NER) tasks. For the purpose of donor relation email parsing.

## Common Definitions from the University of Manitoba

In [1]:
faculties = {
    "Faculty of Agricultural and Food Sciences",
    "School of Agriculture",
    "School of Art",
    "Faculty of Arts",
    "I. H. Asper School of Business",
    "Faculty of Education",
    "Price Faculty of Engineering",
    "Clayton H. Riddell Faculty of Environment, Earth, and Resources",
    "Extended Education",
    "Faculty of Graduate Studies",
    "Rady Faculty of Health Sciences",
    "School of Dental Hygiene",
    "Dr. Gerald Niznick College of Dentistry",
    "Max Rady College of Medicine",
    "College of Nursing",
    "College of Pharmacy",
    "College of Rehabilitation Sciences",
    "Faculty of Kinesiology and Recreation Management",
    "Faculty of Law",
    "Desautels Faculty of Music",
    "Faculty of Science",
    "Faculty of Social Work",
    "University 1"
}

programs = {
    "Accounting",
    "Actuarial Mathematics - Business",
    "Actuarial Mathematics - Science",
    "Aging (interfaculty option)",
    "Agribusiness",
    "Agriculture",
    "Agriculture Diploma",
    "Agroecology",
    "Agronomy",
    "Animal Systems",
    "Anthropology",
    "Applied Mathematics",
    "Architecture (Masters)",
    "Art",
    "Art History (BA)",
    "Art History (BFA)",
    "Asian Studies",
    "Athletic Therapy",
    "Biochemistry",
    "Biological Sciences",
    "Biosystems Engineering",
    "Business",
    "Business Analytics (BComm) (Honours)",
    "Canadian Private Law (Micro-Diploma)",
    "Canadian Public Law (Micro-Diploma)",
    "Canadian Studies",
    "Catholic Studies",
    "Central and East European Studies",
    "Ceramics",
    "Chemistry",
    "City Planning (Masters)",
    "Civil Engineering",
    "Classics",
    "Commerce",
    "Computer Engineering",
    "Computer Science",
    "Criminology",
    "Data Science",
    "Dental Hygiene (BScDH)",
    "Dental Hygiene (Diploma)",
    "Dentistry (BSc)",
    "Dentistry (DMD)",
    "Dentistry (DMD/PhD)",
    "Drawing",
    "Earth Sciences",
    "Economics",
    "Education - Bachelor of Education",
    "Education - Post Baccalaureate",
    "Electrical Engineering",
    "Engineering",
    "English",
    "Entrepreneurship/Small Business",
    "Environmental Design",
    "Environmental Geoscience (BSc)",
    "Environmental Science",
    "Environmental Studies",
    "Essentials in Advanced Patient Care for Pharmacists (Micro-Certificate)",
    "Film Studies",
    "Finance",
    "Fine Arts",
    "Food Science",
    "French",
    "Genetics",
    "Geography",
    "Geology",
    "Geophysics",
    "German",
    "German Language, Life and Culture (Micro-Diploma)",
    "Global Political Economy",
    "Graphic Design",
    "Greek",
    "Health Sciences",
    "Health Studies",
    "History",
    "Human Nutritional Sciences",
    "Human Resource Management / Industrial Relations",
    "Icelandic",
    "Indigenous Business Studies",
    "Indigenous Governance",
    "Indigenous Languages",
    "Indigenous Studies",
    "Interior Design (Masters)",
    "International Business",
    "International Dentist Degree Program",
    "International Medical Graduate Program",
    "Italian",
    "Jazz Studies",
    "Judaic Studies",
    "Juris Doctor (JD)",
    "Labour Relations and Workplace Studies (Diploma)",
    "Labour Studies",
    "Landscape Architecture (Masters)",
    "Latin",
    "Latin American Studies",
    "Law",
    "Leadership and Organizations",
    "Leadership for Business and Organizations (Minor)",
    "Linguistics",
    "Logistics and Supply Chain Management",
    "Management",
    "Management Information Systems",
    "Management Minor for Non-Business Students",
    "Marketing",
    "Mathematics",
    "Mechanical Engineering",
    "Medicine",
    "Medicine (BSc Med)",
    "Medieval and Early Modern Studies",
    "Microbiology",
    "Midwifery",
    "Music",
    "Music (Post-Baccalaureate Diploma in Performance)",
    "Music Minor",
    "Mythology and Folktale (Micro-Diploma)",
    "Nursing",
    "Occupational Therapy (Masters)",
    "Painting",
    "Pharmacy",
    "Philosophy",
    "Photography",
    "Physical Education",
    "Physical Geography",
    "Physics and Astronomy",
    "Physiology and Pathophysiology (postbaccalaureate diploma)",
    "Plant Biotechnology",
    "Polish (Minor)",
    "Political Studies",
    "Pre-Veterinary Medicine",
    "Print Media",
    "Psychology - Arts",
    "Psychology - Science",
    "Recreation Management and Community Development",
    "Recreation Studies (Minor)",
    "Religion",
    "Respiratory Therapy",
    "Russian",
    "Science",
    "Sculpture",
    "Social Work – Distance Delivery",
    "Social Work – Fort Garry campus",
    "Social Work – Inner City Program at William Norrie Centre",
    "Social Work – Northern Program in Thompson",
    "Sociology",
    "Songmaking (Micro-Certificate)",
    "Spanish",
    "Sport, Physical Activity and Recreation in the Community Certificate (SPARC)",
    "Statistics",
    "Strategy and Global Management (BComm) (Honours)",
    "Strategy and Global Management",
    "Theatre",
    "Ukrainian",
    "Ukrainian Canadian Heritage Studies",
    "University 1",
    "Video",
    "Women's and Gender Studies",
    "Workplace Health and Safety (Micro-Diploma)"
}

colleges = {
    "St. Andrew’s College",
    "St. John’s College",
    "St. Paul’s College",
    "University College"
}

reading_levels = {
    "grade 7": "Uses simple sentences and informal tone.",
    "grade 9": "Uses moderate complexity with some formal tone.",
    "grade 12": "Uses formal language, structured paragraphs, and advanced vocabulary."
}

payment_methods = { 
    "cash", "wire transfer", "credit card", "cheque", "gifts in kind"
}

distribution = {
    "bursary", "scholarship", "award", "fellowship", "prize"
}


# Importing OLLAMA
For this we'll use locally hosted models to generate synthtic data


In [2]:
import ollama

## Prompt Generation Function
We'll use a template function to generate the prompts to feed into the LLM to get some more radnomized results from the models.

In [ ]:
import random

def create_json_prompt_for_synthetic_data(**kwargs):
    """
    Generates a synthetic data generation prompt for donor relations emails,
    incorporating structured faculty, program, college, payment methods, and distribution types dynamically.
    """
    attributes = {key: value for key, value in kwargs.items() if value != "n/a"}

    # Randomly select values from faculties, programs, colleges, payment methods, and distributions
    faculty = random.choice(list(faculties))
    program = random.choice(list(programs))
    college = random.choice(list(colleges))
    reading_level, tone_instruction = random.choice(list(reading_levels.items()))
    payment_method = random.choice(list(payment_methods))
    distribution_type = random.choice(list(distribution))

    # Define the prompt structure
    prompt = f"""
You are a synthetic data generator tasked with producing realistic donor relations email passages. Each email should include clearly identified named entities and reflect communications from potential donors to university advancement or giving offices regarding various forms of financial support.

**Objective:**
- Generate human-like emails that vary in tone, sentence structure, and formatting.
- Each email should include multiple annotated entities, covering donor names, contact details, donation amounts, and more.
- Incorporate dynamic structured data: faculty, program, college, payment methods, and distribution types.
- The email should feel natural and realistic—include greetings, optional subject lines, and varied closings.

**Format Requirements:**
- Output must be valid JSON with two keys: "text" for the email content and "entities" for the annotations.
- Every structured piece of information (e.g., {faculty}, {program}, {college}, {payment_method}, {distribution_type}) must appear both in the email text and be annotated in the "entities" list.
- Use realistic donation amounts and formats (e.g., "$5,000", "$5000.00").

**Entity Annotation Guidelines:**
- All entity types must be in lowercase (e.g., "name", "email address").
- Use multiword labels when necessary (e.g., "payment methods", "installment initial date").
- Entities may be nested or span multiple types—list all relevant types.
- Ensure annotations are precise and correspond to text segments.

**Writing Style Requirement:**
- The reading level is **{reading_level}**.
- **{tone_instruction}**
- Keep each email 150-300 words long for readability

**Additional Variability Guidelines:**
- Vary the email structure: include subject lines, different greetings (e.g., "Dear", "Hello", "Hi"), and varied closings.
- Mix detailed and succinct sentences.
- Integrate contact information (email, phone, address) naturally.
- Mention the college dynamically to provide context on the affiliation.

**Output JSON Schema:**
```json
{{
  "text": "generated email text",
  "entities": [
    {{
      "entity": "entity value",
      "types": ["type one", "type two"]
    }}
  ]
}}
```

### **Example 1 (Grade 12 Reading Level - Formal)**
```json
{{ "text": "Subject: Contribution Inquiry - {faculty} at {college}\\n\\nDear Ms. Catherine Reynolds,\\n\\nI hope this email finds you well. My name is Jonathan Smith, and I am interested in making a contribution to the {faculty} at {college}. Specifically, I would like to establish an annual pledge of $5,000 CAD to support {distribution_type} initiatives within the {program} program. \\n\\nI would prefer to make payments via {payment_method}, with the first installment scheduled for June 1, 2025. Please let me know the next steps and any paperwork I need to complete. You can reach me at jonathan.smith@email.com or (204) 555-6789.\\n\\nBest regards,\\nJonathan Smith\\n100 Main Street, Winnipeg, MB R3C 1A3, Canada", "entities": [ {{"entity": "Catherine Reynolds", "types": ["name"]}}, {{"entity": "Jonathan Smith", "types": ["name"]}}, {{"entity": "{faculty}", "types": ["faculty"]}}, {{"entity": "{program}", "types": ["program"]}}, {{"entity": "{college}", "types": ["college"]}}, {{"entity": "{distribution_type}", "types": ["distribution"]}}, {{"entity": "jonathan.smith@email.com", "types": ["email address"]}}, {{"entity": "(204) 555-6789", "types": ["phone number"]}}, {{"entity": "100 Main Street", "types": ["address"]}}, {{"entity": "Winnipeg", "types": ["city"]}}, {{"entity": "MB", "types": ["province"]}}, {{"entity": "R3C 1A3", "types": ["postal code"]}}, {{"entity": "Canada", "types": ["country"]}}, {{"entity": "{payment_method}", "types": ["payment methods"]}}, {{"entity": "CAD", "types": ["currency"]}}, {{"entity": "June 1, 2025", "types": ["installment initial date"]}}, {{"entity": "annual pledge", "types": ["gift type"]}}, {{"entity": "5,000", "types": ["money"]}} ] }}
```
### **Example 2 (Grade 7 Reading Level - Informal)**
```json
{{ "text": "Subject: Donation Question\\n\\nHey Liz,\\n\\nHope ur doin good! I've been thinkin bout donating $5000 to the {faculty} at {college} so that students in the {program} can get extra support through {distribution_type}. Not sure how to start tho—should I fill out a form or just send the money using {payment_method}?\\n\\nLemme know. You can reach me at robert.mitchell@email.ca.\\n\\nCheers,\\nRob", "entities": [ {{"entity": "Liz", "types": ["name"]}}, {{"entity": "Robert Mitchell", "types": ["name"]}}, {{"entity": "{faculty}", "types": ["faculty"]}}, {{"entity": "{program}", "types": ["program"]}}, {{"entity": "{college}", "types": ["college"]}}, {{"entity": "{distribution_type}", "types": ["distribution"]}}, {{"entity": "robert.mitchell@email.ca", "types": ["email address"]}}, {{"entity": "{payment_method}", "types": ["payment methods"]}}, {{"entity": "5000", "types": ["money"]}} ] }}
```
"""
    return prompt


# Now we will start to generate all of the prompts to feed into the LLM


In [13]:
result = create_json_prompt_for_synthetic_data()
print(result)


You are a synthetic data generator tasked with producing realistic donor relations email passages. Each email should include clearly identified named entities and reflect communications from potential donors to university advancement or giving offices regarding various forms of financial support.

**Objective:**
- Generate human-like emails that vary in tone, sentence structure, and formatting.
- Each email should include multiple annotated entities, covering donor names, contact details, donation amounts, and more.
- Incorporate dynamic structured data: faculty, program, college, payment methods, and distribution types.
- The email should feel natural and realistic—include greetings, optional subject lines, and varied closings.

**Format Requirements:**
- Output must be valid JSON with two keys: "text" for the email content and "entities" for the annotations.
- Every structured piece of information (e.g., Clayton H. Riddell Faculty of Environment, Earth, and Resources, Actuarial Math

## Define the data model
This is the data format the the model is told to follow so we will use OLLama
to properly format the data into this format.

In [ ]:
from pydantic import BaseModel
from typing import List

class Entity(BaseModel):
    entity: str
    types: List[str]

class Result(BaseModel):
    text: str
    entities: List[Entity]

## Calling Model
Now, we'll call mistral and opensource 7b model to generate the data.


In [ ]:
for prompt in all_prompts:
    
    resutlt = ollama.chat(
        messages=[{'role': 'user', 'content': prompt}],
        model="mistral:latest",
        format=Result.model_json_schema(),
    )
    print(resutlt)

